In [ ]:
import os
from PIL import Image
from shutil import copyfile
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

#wd = os.getcwd()
#print("working directory is ", wd)

# Hier den Dateipfard eingeben in dem sich der Ordner befindet <------------------ Aktion erforderlich
# Dort muss auch die chromedriver.exe liegen
#---------------------------------------------------------------
dataPath = 'C:/Users/Smurf/Bildertest/'
#---------------------------------------------------------------
#---------------------------------------------------------------E:/08_Projekte/Unternehmenssoftware/picturesjapan/
#building an art-piece dictionary
names = ["michelangelo_buonarotti", "vincent_van_gogh", "edvard_munch"]
pieces = {"michelangelo_buonarotti": ["die_heilige_familie", "die_erschaffung_adams"], 
          "vincent_van_gogh": ["das nachtcafe","gemuesegaerten auf dem montmartre"], 
          "edvard_munch": ["der_schrei", "der_kuss"]}
#copyfile('./chromedriver.exe', dataPath)

for dkey, dvalues in pieces.items():
    for lvalue in dvalues:
        element = dkey +' '+ lvalue
        print(element)
        
        #opens chrome and confirms conditions
        driver = webdriver.Chrome(executable_path = 'C:/Users/Smurf/Desktop/chromedriver')
        driver.get('https://www.google.de/imghp')
        button = driver.find_element_by_css_selector('#L2AGLb > div')
        button.click()
        
        #types subject in search field
        box = driver.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input')
                
        #element = 'edvard munch der schrei'
        box.send_keys(element)
        box.send_keys(Keys.ENTER)
        
        #open style "original"
        button = driver.find_element_by_css_selector('#i6 > div.FAZ4xe.FoDaAb > span > span:nth-child(1) > div > a')
        button.click()
        
        #Will keep scrolling down the webpage until it cannot scroll no more
        last_height = driver.execute_script('return document.body.scrollHeight')
        while True:
            driver.execute_script('window.scrollTo(0,document.body.scrollHeight)')
            new_height = driver.execute_script('return document.body.scrollHeight')
            try:
                driver.find_element_by_xpath('//*[@id="islmp"]/div/div/div/div/div[5]/input').click()
            except:
                pass
            if new_height == last_height:
                break
            last_height = new_height
            
        #checks if folder exists, if not create it
        folder = ['test', 'valid', 'train']
        folders = ['valid', 'train']

        os.chdir(dataPath)
        if not os.path.exists(folder[0]):
            os.makedirs(folder[0])
    
        os.chdir(dataPath)
        if not os.path.exists(folder[1]):
            os.makedirs(folder[1])
            
        os.chdir(dataPath)
        if not os.path.exists(folder[2]):
            os.makedirs(folder[2])
        
        for s in folders:
            os.chdir(''+dataPath+''+s+'/')
            if not os.path.exists(''+dkey+'-'+lvalue+'/'):
                os.makedirs(''+dkey+'-'+lvalue+'/')
        
                      
        #takes screenshots from every picture in the range    
        for i in range(1,15):
            try:
                driver.find_element_by_tag_name('body').send_keys(Keys.CONTROL + Keys.HOME)
                time.sleep(0.2)
                driver.find_element_by_xpath('//*[@id="islrg"]/div[1]/div['+str(i)+']/a[1]/div[1]/img').screenshot(''+dataPath+''+str(i)+'.png')
            except:
                pass
           
        os.chdir(''+dataPath+'')
        driver.close()
        
        #resizes all pictures
        for j in range(1,15):
            PATH = ''+str(j)+'.png'
            if os.path.isfile(PATH) and os.access(PATH, os.R_OK):
                if j <= 10:
                    testtrain = 'train/'
                elif j <= 13: 
                    testtrain = 'valid/'
                else:
                    testtrain = 'test/'
                image = Image.open(''+str(j)+'.png')
                new_image = image.resize((224, 224))
                rgb_new_image = new_image.convert('RGB')
                if testtrain == 'test/':
                    rgb_new_image.save(''+testtrain+'/'+lvalue+'.jpg')
                    #i = i + 1
                else:
                    rgb_new_image.save(''+testtrain+'/'+dkey+'-'+lvalue+'/'+dkey+'-'+lvalue+''+str(j)+'.jpg')
                os.remove(''+str(j)+'.png')
            else:
                print("Either the file is missing or not readable")

# Importieren der notwendigen Module

In [1]:
import os
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam, RMSprop
import numpy as np
from keras.preprocessing import image

# Funktion fuer den Bildtitel mit Kuenstler und Aufruf von "wikiAufruf" 

In [2]:
def Bildtext(string):
    for x, y in train_generator.class_indices.items():
        if y == np.argmax(string):
            print(x)
            maler,bild = x.split('-')
            wikiAufruf(maler)

# Funktion fuer die Vorhersage eines geladenen Bildes

In [3]:
def Bildvorhersage(path):
    img = image.load_img(path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)

    images = np.vstack([x])
    classes = model.predict(images, batch_size=10)
    test = np.argmax(classes)
    if test < np.where(np.isclose(classes, 1.2)):
        print("Bild ist nicht bekannt!")
    else:    
        Bildtext(classes)

# Funktion fuer den Aufruf des Kuenstlers in Wikipedia

In [4]:
from selenium import webdriver
import time

def wikiAufruf(painter):
    chrome_driver = webdriver.Chrome(executable_path = 'C:/Users/Smurf/Desktop/chromedriver')
    chrome_driver.get('https://wikipedia.org')
    chrome_driver.maximize_window()
    search_field = chrome_driver.find_element_by_name('search')
    search_field.send_keys(painter)
    button = chrome_driver.find_element_by_css_selector('#search-form > fieldset > button > i')
    button.click()
    time.sleep(1000)

# Benutzung des VGG16 Models ohne "Spitze"

In [5]:
vgg16_model = VGG16(include_top = False,input_shape=(224,224,3))
#vgg16_model.summary()

# Ergaenzung des VGG16-Rumpfes mit eigenen Layern

In [6]:
model = Sequential()
model.add(vgg16_model)
model.add(Flatten())
model.add(Dense(4096, activation="relu"))
model.add(Dense(1024, activation="relu"))
model.add(Dense(6, activation="softmax"))

# Einlesen der Trainingsdaten im Format 224x224

In [7]:
TRAINING_DIR = "C:/Users/Smurf/Bildertest/train"
training_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = training_datagen.flow_from_directory(
    TRAINING_DIR,
    target_size = (224,224),
    class_mode = 'categorical'
)

Found 60 images belonging to 6 classes.


# Einlesen der Validierungsdaten im Format 224x224

In [8]:
VALIDATION_DIR = "C:/Users/Smurf/Bildertest/valid"
validation_datagen = ImageDataGenerator(rescale = 1./255)
validation_generator = validation_datagen.flow_from_directory(
    VALIDATION_DIR,
    target_size = (224,224),
    class_mode = 'categorical'
)

Found 18 images belonging to 6 classes.


# Kompilierung des Models mit Optimizer "Adam", learning_rate = 0.00001

In [9]:
model.compile(optimizer=Adam(learning_rate=0.00001),loss = "categorical_crossentropy", metrics = ["acc"])

# trainieren des Models mit 3 Epochen

In [ ]:
model.fit(train_generator, epochs = 2,
    validation_data = validation_generator,
    verbose = 1)

Epoch 1/2


In [ ]:
Bildvorhersage('7.jpg')